In [34]:
import mysql.connector as sql
import glob
import pandas as pd
from datetime import datetime

In [35]:
df = pd.read_csv("/home/garage/workspace/price_update_crontab/upbit_krwxrp_1min.csv")

In [36]:
df = df.tail(11)
print(df)

                timestamp   open  close   high    low        volume
1566  2020-09-18T09:54:00  295.0  295.0  295.0  294.0  10196.197525
1567  2020-09-18T09:55:00  294.0  295.0  295.0  294.0   2346.182616
1568  2020-09-18T09:56:00  294.0  295.0  295.0  294.0    206.311812
1569  2020-09-18T09:57:00  294.0  295.0  295.0  294.0  15257.326478
1570  2020-09-18T09:58:00  294.0  295.0  295.0  294.0   9333.341988
1571  2020-09-18T09:59:00  295.0  294.0  295.0  294.0  30251.136929
1572  2020-09-18T10:00:00  294.0  295.0  295.0  294.0   8732.218519
1573  2020-09-18T10:01:00  295.0  295.0  295.0  294.0   8064.359638
1574  2020-09-18T10:02:00  295.0  295.0  295.0  294.0  37692.301416
1575  2020-09-18T10:03:00  295.0  295.0  295.0  295.0      0.595555


In [37]:
db_connection = sql.connect(host='root.cqyptexqvznx.ap-northeast-2.rds.amazonaws.com',port=int(3306), database='garage_test', user='root', password='koscom!234')
db_cursor = db_connection.cursor()

In [38]:

query = 'SELECT max(base_time) FROM history_prc_upbit WHERE base_dt = %s and coin_type = %s'
db_cursor.execute(query,(datetime.today().strftime("%Y%m%d"),'xrp',))
print(datetime.today().strftime("%Y%m%d"))
max_time = db_cursor.fetchone()[0]
print(max_time)


20200918
10:03:00


In [39]:
for i in range(len(df)):
    tmp = str(df.iloc[i]['timestamp']).replace("-","")
    tmp = tmp.split('T')
    base_dt = tmp[0] 
    coin_type = 'xrp'
    base_time = tmp[1]
    if max_time >= tmp[1] :
        continue  
        
    open_price = df.iloc[i]['open']
    close_price = df.iloc[i]['close']
    high_price = df.iloc[i]['high']
    low_price = df.iloc[i]['low']
    volumn = df.iloc[i]['volume']

    query = "INSERT INTO history_prc_upbit VALUES('{}','{}','{}',{},{},{},{},{})".format(base_dt,base_time,coin_type,open_price,close_price,high_price,low_price,volumn)
    db_cursor.execute(query)
    print(query)
    
db_connection.commit()
db_connection.close()